In [1]:
# Step 1: Install kaggle library
!pip install -q kaggle

# Step 2: Upload kaggle.json (API token file)
from google.colab import files
files.upload()  # ← This will prompt you to upload kaggle.json

# Step 3: Move kaggle.json to the correct location
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Step 4: Download the dataset
!kaggle datasets download -d paramaggarwal/fashion-product-images-small

# Step 5: Unzip the dataset
!unzip -q fashion-product-images-small.zip

# Step 6: Load styles.csv using pandas
import pandas as pd

styles_df = pd.read_csv("styles.csv", on_bad_lines='skip')
print("Shape:", styles_df.shape)
styles_df.head()


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small
License(s): MIT
Shape: (44424, 10)


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [2]:
import pandas as pd
import os

# Load styles.csv
styles_df = pd.read_csv("styles.csv", on_bad_lines='skip')

# Drop rows with missing image links or masterCategory
styles_df = styles_df.dropna(subset=['id', 'masterCategory'])

# Convert id to string and append .jpg to match filenames in the images folder
styles_df['image'] = styles_df['id'].astype(str) + ".jpg"

# Create a dictionary: image filename → masterCategory
image_label_dict = dict(zip(styles_df['image'], styles_df['masterCategory']))

# Optional: Show a few mappings
for i, (img, label) in enumerate(image_label_dict.items()):
    print(f"{img} → {label}")
    if i == 4:
        break


15970.jpg → Apparel
39386.jpg → Apparel
59263.jpg → Accessories
21379.jpg → Apparel
53759.jpg → Apparel


In [3]:
# Get unique masterCategory labels
unique_labels = styles_df['masterCategory'].dropna().unique()

# Count of unique labels
num_labels = len(unique_labels)

print(f"Number of unique masterCategory labels: {num_labels}")
print("Labels:", unique_labels)

Number of unique masterCategory labels: 7
Labels: ['Apparel' 'Accessories' 'Footwear' 'Personal Care' 'Free Items'
 'Sporting Goods' 'Home']


In [4]:
import os
import shutil
from tqdm import tqdm

# Create a root dataset folder
os.makedirs("fashion_data_mas", exist_ok=True)

# Create subfolders for each masterCategory label
for label in styles_df['masterCategory'].unique():
    os.makedirs(os.path.join("fashion_data_mas", label), exist_ok=True)

# Move images into label folders
source_dir = "images"  # Folder from the dataset where original images are stored
for idx, row in tqdm(styles_df.iterrows(), total=len(styles_df)):
    img_name = row['image']
    label = row['masterCategory']
    src = os.path.join(source_dir, img_name)
    dst = os.path.join("fashion_data_mas", label, img_name)
    if os.path.exists(src):
        shutil.copy(src, dst)


100%|██████████| 44424/44424 [00:22<00:00, 1972.92it/s]


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.models import resnet18
torch.cuda.is_available()

True

In [6]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load dataset
full_dataset = datasets.ImageFolder("fashion_data_mas", transform=transform)

# Split dataset: 70% train, 15% val, 15% test
total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size  # remainder for test

# Ensure reproducible splits
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = resnet18(pretrained=True)
model.fc = nn.Linear(512, 7)
model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 173MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [10]:
import torch
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=10, class_names=None):
    best_f1 = 0.0

    for epoch in range(epochs):
        # ---- Training ----
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        f1 = f1_score(all_labels, all_preds, average='weighted')

        # ---- Save Best Model ----
        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), "best_model.pth")
            print(f"✅ Best model updated at Epoch {epoch+1} | F1: {f1:.4f}")

        # ---- Confusion Matrix ----
        cm = confusion_matrix(all_labels, all_preds)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_names if class_names else np.unique(all_labels),
                    yticklabels=class_names if class_names else np.unique(all_labels))
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.title(f'Confusion Matrix - Epoch {epoch+1}')
        plt.tight_layout()
        plt.savefig(f'confusion_matrix_epoch_{epoch+1}.png')
        plt.close()

        # ---- Logging ----
        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | "
              f"Val Loss: {avg_val_loss:.4f} | F1 Score: {f1:.4f}")


In [11]:
train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    epochs=10
)


✅ Best model updated at Epoch 1 | F1: 0.9907
Epoch 1 | Train Loss: 0.0114 | Val Loss: 0.0469 | F1 Score: 0.9907
Epoch 2 | Train Loss: 0.0108 | Val Loss: 0.0487 | F1 Score: 0.9900
✅ Best model updated at Epoch 3 | F1: 0.9913
Epoch 3 | Train Loss: 0.0085 | Val Loss: 0.0498 | F1 Score: 0.9913
Epoch 4 | Train Loss: 0.0072 | Val Loss: 0.0527 | F1 Score: 0.9904
Epoch 5 | Train Loss: 0.0061 | Val Loss: 0.0526 | F1 Score: 0.9900
✅ Best model updated at Epoch 6 | F1: 0.9929
Epoch 6 | Train Loss: 0.0070 | Val Loss: 0.0408 | F1 Score: 0.9929
Epoch 7 | Train Loss: 0.0067 | Val Loss: 0.0522 | F1 Score: 0.9895
Epoch 8 | Train Loss: 0.0061 | Val Loss: 0.0489 | F1 Score: 0.9911
Epoch 9 | Train Loss: 0.0075 | Val Loss: 0.0502 | F1 Score: 0.9918
Epoch 10 | Train Loss: 0.0047 | Val Loss: 0.0606 | F1 Score: 0.9899


In [12]:
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F
import copy

In [13]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='macro')
print(f"\n📊 Test Accuracy: {acc:.4f}, F1 (macro): {f1:.4f}")


📊 Test Accuracy: 0.9938, F1 (macro): 0.7419


In [14]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')
print(f"\n📊 Test Accuracy: {acc:.4f}, F1 (weighted): {f1:.4f}")


📊 Test Accuracy: 0.9938, F1 (weighted): 0.9933
